In [1]:
import ee
import geemap
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
matplotlib.rcParams['font.sans-serif'] = ['SimHei']  # 指定默认字体
matplotlib.rcParams['axes.unicode_minus'] = False  # 解决保存图像时负号'-'显示为方块的问题
ee.Authenticate()
ee.Initialize(project='ee-hyf7753')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [2]:
def mask_s2_clouds(image):
  qa = image.select('QA60')
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )
  return image.updateMask(mask).divide(10000)

In [15]:
polygon_2023 = ee.Geometry.Polygon(
    [[[108.09458101775786, 34.30562265948671], [108.09501064847578, 34.30557687261404], [108.09493276022344, 34.30484602589277], [108.09450312950553, 34.304891812765455], [108.09458101775786, 34.30562265948671]]],
    None, False
)
polygon_2024 = ee.Geometry.Polygon(
    [[[108.09458928029343, 34.30560245447279], [108.09502152807725, 34.30557335942607], [108.0949507876291, 34.30483217493664], [108.0945185436662, 34.30486126992337], [108.09458928029343, 34.30560245447279]]],
    None, False
)

dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    # .filterDate('2023-01-01', '2023-05-30')
    .filterDate('2024-01-01', '2024-05-30')
    .filterBounds(polygon_2024)  # 应用空间过滤器
    # Pre-filter to get less cloudy granules.
    # .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    # .map(mask_s2_clouds)
    .map(lambda img: img.set('date', ee.String(img.get('system:index')).slice(0, 8)))
    .map(lambda image: image.addBands(image.normalizedDifference(["B5", "B4"]).rename("NDVI")))
    .select("NDVI")
    .sort('date')
)

timestamplist = (dataset.aggregate_array('date')
                 .map(lambda d: ee.String(d))
                 .getInfo())

timestamplist_uni = list(set(timestamplist))

In [ ]:
img = dataset.reduce(ee.Reducer.mean()).rename("Mean_NDVI")
print(img.getInfo())
# 感觉这是全部平均也不是单个平均呀

{'type': 'Image', 'bands': [{'id': 'Mean_NDVI', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [18]:
m = geemap.Map()
# 定位和缩放到 Polygon 所在的区域
m.centerObject(polygon_2024, zoom=17)  # 可以调整 zoom 级别以适合你的需求
# 设置卫星影像底图
m.add_basemap('SATELLITE')
# 添加平均NDVI影像到地图
viz_params = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
m.addLayer(img, viz_params, 'Mean NDVI')
m.addLayer(polygon_2024, {'color': 'FF0000'}, 'Polygon Layer')
m

Map(center=[34.3052173149537, 108.09477003504149], controls=(WidgetControl(options=['position', 'transparent_b…